# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [2]:
# some imports

import sys
sys.path.insert(0, "../modules")
import pandas as pd
import cufflinks as cf
import plotly.express as px
import re
import module as mod # functions are include in module.py.
cf.go_offline()

In [3]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [9]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [10]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [42]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica
...,...,...
112,098,Suicidio y lesiones autoinfligidas
113,099,Agresiones (homicidio)
114,100,Eventos de intención no determinada
115,101,Complicaciones de la atención médica y quirúrgica


In [ ]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())

#048: Drogas
#099: Homicidios

## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [154]:
# Example 1

dataset_sexo = mod.row_filter(deaths, 'Sexo', ['Hombres','Mujeres']) #Nos qudamos con hombres y mujeres, quitando el total
dataset_sexo = mod.row_filter(dataset_sexo, 'Edad', ['Todas las edades']) #Nos quedamos con todas las edades.
dataset_sexo = mod.nrow_filter(dataset_sexo, 'cause_name', ['I-XXII.Todas las causas']) #Quitamos todas las causas del motivo de la muerte
dataset_sexo


,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1991,72519,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
1,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1998,71916,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
2,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1999,71792,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
3,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1993,71581,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
4,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1990,71094,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
...,...,...,...,...,...,...,...,...
9043,078 Enfermedades de los órganos genitales mas...,Mujeres,Todas las edades,2009,0,078,Single cause,Enfermedades de los órganos genitales masculinos
9044,078 Enfermedades de los órganos genitales mas...,Mujeres,Todas las edades,2008,0,078,Single cause,Enfermedades de los órganos genitales masculinos
9045,078 Enfermedades de los órganos genitales mas...,Mujeres,Todas las edades,2018,0,078,Single cause,Enfermedades de los órganos genitales masculinos
9046,078 Enfermedades de los órganos genitales mas...,Mujeres,Todas las edades,1988,0,078,Single cause,Enfermedades de los órganos genitales masculinos


In [180]:
dataset_sexo_sorted=dataset_sexo.sort_values('Total').groupby(['Periodo','Sexo']).tail(1) #Nos quedamos con la linea con el total mas grande (la última)
dataset_sexo_periodo=dataset_sexo_sorted.sort_values('Periodo', ascending=True)#Ordenamos la tabla por período
dataset_sexo_periodo.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
36,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1980,66766,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
65,053-061 IX.Enfermedades del sistema circulatorio,Hombres,Todas las edades,1980,59121,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
26,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1981,67918,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
62,053-061 IX.Enfermedades del sistema circulatorio,Hombres,Todas las edades,1981,59668,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio
39,053-061 IX.Enfermedades del sistema circulatorio,Mujeres,Todas las edades,1982,66054,053-061,Multiple causes,IX.Enfermedades del sistema circulatorio


In [197]:

fig = px.bar(dataset_sexo_periodo, x="Periodo", y="Total", barmode="group", color="Sexo",
                            category_orders={"Sexo": ["Hombres", "Mujeres"]})
fig

In [151]:
# Example 2
'''
group = ['cause_code','Periodo']
dataset = mod.groupby_sum(deaths, group)
dataset.head()
'''

"\ngroup = ['cause_code','Periodo']\ndataset = mod.groupby_sum(deaths, group)\ndataset.head()\n"

In [38]:

group_sexo = ['Sexo','cause_code']
dataset_sexo = mod.groupby_sum(deaths, group_sexo)
dataset_sexo

,Sexo,cause_code,Total
0,Total,001-102,27827216
1,Hombres,001-102,14528688
2,Mujeres,001-102,13298528
3,Total,053-061,9802974
4,Total,009-041,7118864
...,...,...,...
346,Hombres,024,0
347,Mujeres,078,0
348,Mujeres,029,0
349,Mujeres,028,0


In [47]:
# Example 3

dataset_periodo = mod.pivot_table(dataset, 'cause_code', 'Periodo')
dataset_periodo.head()

cause_code,Periodo,001,001-008,001-102,002,003,004,005,006,007,...,093,094,095,096,097,098,099,100,101,102
0,1980,1620,15768,1157376,5904,2008,3448,436,0,0,...,4956,1432,184,692,16748,6608,1496,28,968,96
1,1981,1404,15124,1173544,6332,1656,3344,348,0,0,...,4700,1200,156,1396,17472,6872,1284,336,908,208
2,1982,1308,13488,1146620,5352,1240,3104,316,0,0,...,4864,956,200,1000,18616,7404,1228,440,1132,52
3,1983,1212,13100,1210276,5152,1072,3152,336,0,0,...,4788,1464,148,884,18392,8724,1560,1276,1500,56
4,1984,1228,12928,1197636,4564,964,3704,424,0,0,...,4716,1244,164,1020,14696,9972,1812,1144,1636,76


In [49]:
dataset_sexo = mod.pivot_table(dataset_sexo, 'cause_code', 'Sexo')
dataset_sexo.head()

cause_code,Sexo,001,001-008,001-102,002,003,004,005,006,007,...,093,094,095,096,097,098,099,100,101,102
0,Hombres,13310,288068,14528688,37942,3860,89660,24062,92868,2546,...,83108,10350,33694,10804,143376,181592,20420,8170,14920,2008
1,Mujeres,20734,204418,13298528,15510,4110,100022,21674,21704,596,...,51050,6978,8680,5370,70878,60150,8040,3420,14192,838
2,Total,34044,492486,27827216,53452,7970,189682,45736,114572,3142,...,134158,17328,42374,16174,214254,241742,28460,11590,29112,2846


## ...and finally, show me some insights with Plotly!!!

Of course, you can always check the [class notes](https://github.com/ih-datapt-mad/dataptmad1121_lessons/tree/main/module-2) for further info. 

In [23]:
# Cufflinks histogram

dataset.iplot(kind='hist',
                     title='VIZ TITLE',
                     yTitle='AXIS TITLE',
                     xTitle='AXIS TITLE')


In [50]:
# Cufflinks bar plot

dataset_sexo.iplot(kind='bar',
                  x='098',
                  xTitle='AXIS TITLE',
                  yTitle='AXIS TITLE',
                  title='VIZ TITLE')


In [52]:
# Cufflinks line plot

dataset_sexo.iplot(kind='line',
                   x='098',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')


In [54]:
# Cufflinks scatter plot

dataset_sexo.iplot(x='Sexo', 
                      y='098', 
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')


In [112]:

fig = px.bar(dataset1, x="Sexo", y="Total", barmode="group", color="cause_code",
                            category_orders={"Sexo": ["Hombres", "Mujeres"],
                            "cause_code": ["098", "048"]})
fig